In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# --- Parameters ---
grid_size = 64
steps = 100
dopamine_agents_ratio = 0.1  # % of agents under dopamine hijack
coherence_threshold = 0.75   # how aligned neighbors must be to be "coherent"
dopamine_bias = 0.6          # how strongly dopamine agents override field input

# --- Initialization ---
field = np.random.rand(grid_size, grid_size)
dopamine_mask = np.random.rand(grid_size, grid_size) < dopamine_agents_ratio

# --- Simulation functions ---
def update_field(field):
    new_field = field.copy()
    for x in range(grid_size):
        for y in range(grid_size):
            local = field[max(0, x - 1):min(grid_size, x + 2),
                          max(0, y - 1):min(grid_size, y + 2)]
            local_mean = np.mean(local)

            if dopamine_mask[x, y]:
                # Agents hijacked by reward stimulus
                new_field[x, y] = dopamine_bias * np.random.rand() + (1 - dopamine_bias) * local_mean
            else:
                # Recursive coherence-based learning
                new_field[x, y] = 0.5 * field[x, y] + 0.5 * local_mean

    return new_field

def compute_coherence(field):
    diffs = []
    for x in range(grid_size):
        for y in range(grid_size):
            local = field[max(0, x - 1):min(grid_size, x + 2),
                          max(0, y - 1):min(grid_size, y + 2)]
            coherence = 1 - np.std(local)
            diffs.append(coherence)
    return np.mean(diffs)

# --- Animation setup ---
fig, ax = plt.subplots(figsize=(6, 6))

def animate(i):
    global field
    field = update_field(field)
    coherence = compute_coherence(field)
    ax.clear()
    ax.set_title(f"Step {i} — Mean Coherence: {coherence:.3f}")
    ax.imshow(field, cmap="plasma", vmin=0, vmax=1)
    ax.axis("off")

ani = animation.FuncAnimation(fig, animate, frames=steps, interval=200)
ani.save("dopamine_hijack_vs_recursive_learning_sim.gif", writer='pillow')
plt.close()

print("🎥 Simulation saved as 'dopamine_hijack_vs_recursive_learning_sim.gif'")
